[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rM5P-t_nF5s-ZWhbkmgM-vALG-FiYbOn?usp=sharing\)

# Processo de ETL em Python
#### Feito durante Bootcamp Santander 2025 - Ciência de Dados com Python

O presente projeto faz uma busca simples de CPFs num arquivo CSV onde se encontram os CPFs de três clientes fictícios, a partir desses identificadores é feita uma busca na API [RandomUser](https://randomuser.me/) que neste caso gera sempre 4 usuários devido ao uso de seed e outros parâmetros combinados no endpoint da API, no entanto apenas 3 desses clientes serão selecionados conforme dados constantes no CSV.

Diferentemente do projeto de demonstração usado na [DIO](https://www.dio.me/), esta API não disponibiliza um parâmetro para buscar usuários por um ID (sequencial numérico), por isso usei o número do CPF que foi disponibilizado a partir da versão 1.4 da Random User.

Dividi o código em Extract, Transform e Load para facilitar a leitura e localicação no código.
Fiz a extração eliminando a máscara presente nos CPFs do CSV e também da API utilizando Regex para isso (depois que aprendi Regex não parei mais de usar), após isso criei uma nova chave chamada `cpf_value_normalizado` que consiste no CPF sem pontuações. Além disso criei a chave `message` para armazenar a mensagem posteriormente gerada pela IA. Na etapa de Transform faço a comunicação com a API do ChatGPT e adiciono o resultado do Chat na chave `message` para usar na Etapa de Load. Finalmente, na etapa de Load decidi carregar o Nome (RandomUser), E-mail (RandomUser) e a Mensagem gerada pela IA numa planilha Google usando outra API de um serviço chamado [Sheet Monkey](https://sheetmonkey.io/pt-br) que conheci durante um outro curso, fiz isso sem saber se funcionaria, pois havia usado ele em formulários HTML apenas. Ah, enquanto meus períodos de testes estiverem vigentes as APIs funcionarão! 😅

Fazer um processo de ETL foi algo totalmente novo pra mim, principalmente por ser uma das primeiras vezes que estou codando em Python. Tive que pesquisar bastante sobre como poderia usar o recurso da OpenAI no Azure, pois atualmente tenho créditos no Azure para estudar e isso me possibilitou usar o serviço da OpenAI mesmo não tendo créditos na plataforma própria deles. Foram muitas conversas com o Copilot e ChatGPT  até conseguir implementar o recurso na nuvem e fazer funcionar aqui. 😅

🌐🔗 Links:
* Planilha: https://docs.google.com/spreadsheets/d/1N3WdpqJ8oX9GWHQTglTfbs9YytcdWkASQ4LtHLzmfrQ/edit?usp=sharing
* Repositório no GitHub: https://www.linkedin.com/in/bruno-de-paula-teixeira/
* Linkedin: https://www.linkedin.com/in/bruno-de-paula-teixeira/

*\*Não deixei as chaves da API e o endpoint hardcoded por questões de segurança, eles são solicitados no momento da execução da etapa de Transform.*

Por Bruno de Paula Teixeira

### Código 👨‍💻


Extract

In [ ]:
!pip install openai

In [ ]:
import pandas as pd
import re
import requests
import json

df = pd.read_csv('Identificadores.csv') # dataframe
customers_id_csv = df['CUSTOMER_ID'].astype(str).str.replace(r'\D', '', regex=True).tolist()
print(customers_id_csv)

In [ ]:
def get_user(id):
    response = requests.get(f'https://randomuser.me/api/1.4/?results=4&nat=br&seed=brasil') # busca sempre os mesmos 4 usuarios na API RandomUser

    if response.status_code != 200:
        return None

    all_users = response.json()["results"]

    for u in all_users:
        u["id"]["cpf_value_normalizado"] = re.sub(r'\D', '', u["id"]["value"]) # adiciona uma chave com o valor do cpf sem mascara (normalizado)
        u["message"] = "" # onde vai ser armazenada a mensagem da IA

    found_users = next((user for user in all_users if user["id"]["cpf_value_normalizado"] == id), None)
    return found_users

users = [user for nmr_cpf_from_csv in customers_id_csv if (user := get_user(nmr_cpf_from_csv)) is not None] # list comprehension
print(json.dumps(users, indent=3))

Transform

In [ ]:
from openai import AzureOpenAI
azure_openai_api_key = input('Digite sua API key da OpenAI: ')
endpoint = input('Digite o endpoint: ')

client = AzureOpenAI(
    api_key = azure_openai_api_key,
    api_version = "2024-02-01",
    azure_endpoint = endpoint
)

def generate_ai_news(user):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {
            "role": "system",
            "content": "Você é um especialista em marketing bancário."
        },
        {
            "role": "user",
            "content": f"Crie uma mensagem para {user['name']['first']} sobre a importância dos investimentos. {user['name']['first']} é o nome do cliente. (Máximo de 100 caracteres)"
        }
        ]
    )

    return response.choices[0].message.content.strip('\"') # resposta do ChatGPT

for user in users:
    message = generate_ai_news(user)
    user['message'] = message
    print(message)


Load - Insere os dados no Google Sheets: https://docs.google.com/spreadsheets/d/1N3WdpqJ8oX9GWHQTglTfbs9YytcdWkASQ4LtHLzmfrQ/edit?usp=sharing

In [ ]:
def post_user_sheet(json):
    response = requests.post(f"https://api.sheetmonkey.io/form/5UE5uYQ3KDxszByhM9XHCG", json)
    return True if response.status_code == 200 else False
# Insere os dados no Google Sheets: https://docs.google.com/spreadsheets/d/1N3WdpqJ8oX9GWHQTglTfbs9YytcdWkASQ4LtHLzmfrQ/edit?usp=sharing

for user in users:
    json_user = {"Name": user['name']['first'], "Email": user['email'], "Mensagem": user['message'], "Created": "x-sheetmonkey-current-date-time"}
    success = post_user_sheet(json_user)
    print(f"User {user['name']['first']} updated? {success}!")